In [4]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

app = Flask(__name__)
CORS(app)

# Database setup
Base = declarative_base()
engine = create_engine('sqlite:///employees.db')
Session = sessionmaker(bind=engine)
session = Session()

# Employee model
class Employee(Base):
    __tablename__ = 'employees'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    role = Column(String, nullable=False)
    projects = Column(String, nullable=False)
    leaves = Column(Integer, nullable=False)

Base.metadata.create_all(engine)

# Save employee details
@app.route('/save', methods=['POST'])
def save_employee():
    data = request.json
    employee = Employee(
        name=data['name'],
        role=data['role'],
        projects=data['projects'],
        leaves=data['leaves']
    )
    session.add(employee)
    session.commit()
    return jsonify({'message': 'Employee saved successfully!'})

# Retrieve all employees
@app.route('/retrieve', methods=['GET'])
def retrieve_employees():
    employees = session.query(Employee).all()
    result = [
        {
            'id': emp.id,
            'name': emp.name,
            'role': emp.role,
            'projects': emp.projects,
            'leaves': emp.leaves
        }
        for emp in employees
    ]
    return jsonify(result)

# Retrieve details about an employee by name
@app.route('/query', methods=['POST'])
def query_employee():
    name = request.json.get('name')
    employee = session.query(Employee).filter(Employee.name.ilike(f'%{name}%')).first()
    if employee:
        response = {
            'name': employee.name,
            'role': employee.role,
            'projects': employee.projects,
            'leaves': employee.leaves
        }
        return jsonify(response)
    return jsonify({'message': 'Employee not found'}), 404

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'sqlalchemy'